Import Required Libraries


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Importing the dataset

In [4]:
from google.colab import files
uploaded = files.upload()

Saving UCI_Credit_Card.csv to UCI_Credit_Card.csv


In [0]:
defaultCC = pd.read_csv("UCI_Credit_Card.csv")

Some Basic Exploratory Data Analysis

In [19]:
defaultCC.shape

(30000, 25)

In [20]:
defaultCC.dtypes

ID                              int64
LIMIT_BAL                     float64
SEX                             int64
EDUCATION                       int64
MARRIAGE                        int64
AGE                             int64
PAY_0                           int64
PAY_2                           int64
PAY_3                           int64
PAY_4                           int64
PAY_5                           int64
PAY_6                           int64
BILL_AMT1                     float64
BILL_AMT2                     float64
BILL_AMT3                     float64
BILL_AMT4                     float64
BILL_AMT5                     float64
BILL_AMT6                     float64
PAY_AMT1                      float64
PAY_AMT2                      float64
PAY_AMT3                      float64
PAY_AMT4                      float64
PAY_AMT5                      float64
PAY_AMT6                      float64
default.payment.next.month      int64
dtype: object

In [21]:

defaultCC.isnull().sum()

ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default.payment.next.month    0
dtype: int64

In [22]:
defaultCC.head(5)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [23]:
defaultCC["SEX"].value_counts()

2    18112
1    11888
Name: SEX, dtype: int64

In [24]:
defaultCC["EDUCATION"].value_counts()

2    14030
1    10585
3     4917
5      280
4      123
6       51
0       14
Name: EDUCATION, dtype: int64

In [25]:
defaultCC["MARRIAGE"].value_counts()

2    15964
1    13659
3      323
0       54
Name: MARRIAGE, dtype: int64

In [26]:
defaultCC["default.payment.next.month"].value_counts()

0    23364
1     6636
Name: default.payment.next.month, dtype: int64

In [0]:
defaultCC1 = defaultCC.copy()

In [0]:
defaultCC.rename(columns = lambda x:x.lower(), inplace=True)

In [0]:
defaultCC['grad_school'] = (defaultCC['education'] == 1).astype('int')
defaultCC['university'] = (defaultCC['education'] == 2).astype('int')
defaultCC['high_school'] = (defaultCC['education'] == 3).astype('int')
defaultCC.drop(['education'],1,inplace = True)

In [30]:
defaultCC.head(5)

,id,limit_bal,sex,marriage,age,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default.payment.next.month,grad_school,university,high_school
0,1,20000.0,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1,0,1,0
1,2,120000.0,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1,0,1,0
2,3,90000.0,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0,0,1,0
3,4,50000.0,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0,0,1,0
4,5,50000.0,1,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0,0,1,0


In [0]:
defaultCC['male'] = (defaultCC['sex'] == 1).astype('int')
defaultCC.drop(['sex'],1,inplace=True)

In [0]:
defaultCC['married'] = (defaultCC['marriage'] == 1).astype('int')
defaultCC.drop(['marriage'],1,inplace=True)

In [0]:
pay_features = ['pay_0','pay_2','pay_3','pay_4','pay_5','pay_6']
for p in pay_features:
    defaultCC.loc[defaultCC[p]<=0,p] = 0

In [0]:
defaultCC.rename(columns={'default.payment.next.month':'default'}, inplace=True)

In [35]:

defaultCC.head(5)

,id,limit_bal,age,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default,grad_school,university,high_school,male,married
0,1,20000.0,24,2,2,0,0,0,0,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1,0,1,0,0,1
1,2,120000.0,26,0,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1,0,1,0,0,0
2,3,90000.0,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0,0,1,0,0,0
3,4,50000.0,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0,0,1,0,0,1
4,5,50000.0,57,0,0,0,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0,0,1,0,1,1


Modelling

In [0]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, precision_recall_curve
from sklearn.preprocessing import RobustScaler

In [0]:
target_name = 'default'
X = defaultCC.drop(['default'], 1)
robust_scaler = RobustScaler()
X = robust_scaler.fit_transform(X)
y = defaultCC[target_name]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.15, random_state = 123, stratify = y)

In [0]:
def Cmatrix(CM, labels=['pay','default']):
    df = pd.DataFrame(data=CM, index=labels, columns=labels)
    df.index.name = 'TRUE'
    df.columns.name = 'PREDICTION'
    df.loc['Total'] = df.sum()
    df['Total'] = df.sum(axis=1)
    return df

In [0]:
metrics = pd.DataFrame(index=['accuracy','precision','recall'],columns = ['LogisticReg','ClassTree','KNN'])

In [0]:

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [0]:
logistic_regression = LogisticRegression(n_jobs=-1)

In [54]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(logistic_regression, X, y, cv=10,scoring='accuracy')
print (accuracy)
metrics.loc['accuracy','LogisticReg'] =(cross_val_score(logistic_regression, X, y, cv=10,scoring='accuracy').mean())
metrics.loc['precision','LogisticReg']=(cross_val_score(logistic_regression, X, y, cv=10,scoring='precision').mean())
metrics.loc['recall','LogisticReg']=(cross_val_score(logistic_regression, X, y, cv=10,scoring='recall').mean())

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'l

[0.80506498 0.80573142 0.81006331 0.80573142 0.81966667 0.82866667
 0.83027676 0.82527509 0.82627543 0.82227409]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'l

In [56]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree
parameters = {'max_depth':range(3,20)}
clf = GridSearchCV(tree.DecisionTreeClassifier(), parameters, n_jobs=4)
clf.fit(X,y)
tree_model = clf.best_estimator_
print (clf.best_score_, clf.best_params_) 
y_true, y_pred = y, clf.predict(X)
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

metrics.loc['accuracy','ClassTree'] = clf.best_score_
metrics.loc['precision','ClassTree']= 0.76
metrics.loc['recall','ClassTree'] = 0.66

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.8053 {'max_depth': 3}
              precision    recall  f1-score   support

           0       0.84      0.95      0.89     23364
           1       0.68      0.37      0.48      6636

    accuracy                           0.82     30000
   macro avg       0.76      0.66      0.68     30000
weighted avg       0.81      0.82      0.80     30000



In [0]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(metric='euclidean')
knn_model.fit(X_train,y_train)
y_pred_test = knn_model.predict(X_test)
metrics.loc['accuracy','KNN'] = accuracy_score(y_pred=y_pred_test, y_true=y_test)
metrics.loc['precision','KNN'] = precision_score(y_pred=y_pred_test, y_true=y_test)
metrics.loc['recall','KNN'] = recall_score(y_pred=y_pred_test, y_true=y_test)


In [58]:
metrics

,LogisticReg,ClassTree,KNN
accuracy,0.817903,0.8053,0.790667
precision,0.684742,0.76,0.541997
recall,0.330772,0.66,0.343719


In [0]:
runs = 0

In [0]:
def f_metrics(x1,y1,x2,y2,runs):
    
    clf.fit(x1,y1)
    y_pred = clf.predict(x2)
    score = accuracy_score(y2,y_pred)
    modelnumber = 1
    from sklearn.metrics import precision_score
    average_precision = precision_score(y_test, y_pred,average="weighted")
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import recall_score
    cm = confusion_matrix(y_test,y_pred)
    recall = recall_score(y_test, y_pred,average="weighted")
    data = {'score':score,'modelnumber':modelnumber,'runs':runs,"precision":average_precision,"confusion matrix":cm,"recall":recall}
    df1 = pd.DataFrame([data])
    df1.columns = ["confusion Matrix","modelnumber","precision","recall","runs","score"]
    
    logistic_regression.fit(x1,y1)
    y_pred = logistic_regression.predict(x2)
    score1 = accuracy_score(y2,y_pred)
    modelnumber = modelnumber+1
    average_precision = precision_score(y_test, y_pred,average="weighted")
    cm = confusion_matrix(y_test,y_pred)
    recall = recall_score(y_test, y_pred,average="weighted")
    data = {'score':score1,'modelnumber':modelnumber,'runs':runs,"precision":average_precision,"confusion matrix":cm,"recall":recall}
    df2 = pd.DataFrame([data])
    df2.columns = ["confusion Matrix","modelnumber","precision","recall","runs","score"]
    df1 = df1.append(df2)
    
    knn_model.fit(x1,y1)
    y_pred = knn_model.predict(x2)
    score1 = accuracy_score(y2,y_pred)
    modelnumber = modelnumber+1
    average_precision = precision_score(y_test, y_pred,average="weighted")
    cm = confusion_matrix(y_test,y_pred)
    recall = recall_score(y_test, y_pred,average="weighted")
    data = {'score':score1,'modelnumber':modelnumber,'runs':runs,"precision":average_precision,"confusion matrix":cm,"recall":recall}
    df3 = pd.DataFrame([data])
    df3.columns = ["confusion Matrix","modelnumber","precision","recall","runs","score"]
    
    df1 = df1.append(df3)
    df1= df1.reset_index(drop=True)
   
    return df1

In [75]:

runs = runs + 1
final_metrics = f_metrics(X_train,y_train,X_test,y_test,runs)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


In [76]:
final_metrics

,confusion Matrix,modelnumber,precision,recall,runs,score
0,"[[3337, 168], [633, 362]]",1,0.805721,0.822000,2,0.822000
1,"[[3365, 140], [672, 323]]",2,0.803487,0.819556,2,0.819556
2,"[[3216, 289], [653, 342]]",3,0.767272,0.790667,2,0.790667


In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [78]:
defaultCC.shape

(30000, 27)

In [82]:
classifier = Sequential()
classifier.add(Dense(output_dim=10, init='uniform', activation='relu', input_dim=26))
classifier.add(Dropout(p=0.1))
classifier.add(Dense(output_dim=1, init='uniform', activation='sigmoid'))
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.fit(X_train, y_train, batch_size=10,epochs=20)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=26, units=10, kernel_initializer="uniform")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  after removing the cwd from sys.path.


Epoch 1/20
25500/25500 [==============================] - 4s 140us/step - loss: 0.4705 - acc: 0.8075
Epoch 2/20
25500/25500 [==============================] - 3s 113us/step - loss: 0.4460 - acc: 0.8169
Epoch 3/20
25500/25500 [==============================] - 3s 112us/step - loss: 0.4426 - acc: 0.8177
Epoch 4/20
25500/25500 [==============================] - 3s 112us/step - loss: 0.4425 - acc: 0.8169
Epoch 5/20
25500/25500 [==============================] - 3s 113us/step - loss: 0.4409 - acc: 0.8185
Epoch 6/20
25500/25500 [==============================] - 3s 113us/step - loss: 0.4418 - acc: 0.8175
Epoch 7/20
25500/25500 [==============================] - 3s 114us/step - loss: 0.4410 - acc: 0.8170
Epoch 8/20
25500/25500 [==============================] - 3s 112us/step - loss: 0.4403 - acc: 0.8167
Epoch 9/20
25500/25500 [==============================] - 3s 113us/step - loss: 0.4387 - acc: 0.8166
Epoch 10/20
25500/25500 [==============================] - 3s 113us/step - loss: 0.4402 - a

In [0]:
y_pred = classifier.predict(X_test)
pred=[]
for i in range(len(y_pred)):
    if y_pred[i]>=0.5:
        pred.append(1)
    else:
        pred.append(0)